In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [1]:
import pandas as pd
import os
import cv2
import json

In [2]:
damage_labeling = pd.read_csv('damage_labeling_old.csv')
part_labeling = pd.read_csv('part_labeling_old.csv')

In [4]:
label_dam_dir = '../data/Dataset/2.라벨링데이터/damage/'
label_dam = os.listdir(label_dam_dir)

len(label_dam)

67298

In [5]:
damage_labeling.columns
list(damage_labeling.columns)

['index',
 'Scratched',
 'Breakage',
 'Separated',
 'Crushed',
 'total_anns',
 'ran_var',
 'dataset']

In [70]:
### damage_labeling에서 train/test/val 비율 확인 ###

damage_labeling['dataset'].value_counts(normalize= True)

train    0.799914
val      0.100162
test     0.099923
Name: dataset, dtype: float64

In [6]:
### 원본 damage_labeling의 컬럼으로 df 생성하기 ###
dmg_label = pd.DataFrame(columns=list(damage_labeling.columns))

In [7]:
### (Damage_json) 정보로 csv 만들기 ###

import warnings

warnings.filterwarnings(action='ignore')

df = pd.DataFrame(columns = list(damage_labeling.columns))


for i in range(len(label_dam)):
    with open(label_dam_dir + f'{label_dam[i]}') as file:
        part = json.load(file)
        file_name = part['images']['file_name']
        
        #print(len(part['annotations']))
        
        for j in range(len(part['annotations'])):
            damage = part['annotations'][j]['damage']
            if not damage is None:
                if df.loc[df['index'] == file_name].empty:
                    df = df.append(pd.DataFrame([[file_name,1]], columns=['index',damage]), ignore_index=True)
                else:
                    if df.loc[df['index'] == file_name, damage].values[0] >= 0:
                        if df.loc[df['index'] == file_name].empty:
                            df = df.append(pd.DataFrame([[file_name,1]], columns=['index',damage]), ignore_index=True)
                        else:
                            df.loc[df['index'] == file_name, damage] = df.loc[df['index'] == file_name, damage] + 1
                    else:
                        df.loc[df['index'] == file_name, damage] = 1
                        
df['total_anns'] = df.iloc[:,1:-3].sum(axis = 'columns')


train = df.sample(frac = 0.8, random_state= 7)
train['dataset'] = 'train'
remain = df.drop(index = train.index)
test = remain.sample(frac = 0.5, random_state= 7)
test['dataset'] = 'test'
val = remain.drop(index= test.index)
val['dataset'] = 'val'

df = pd.concat([train, test, val], ignore_index=True)
df = df.sort_values(by = 'index')
df.to_csv("damage_labeling.csv", index = False)
df

,index,Scratched,Breakage,Separated,Crushed,total_anns,ran_var,dataset
57737,0000002_as-0036229.jpg,NaN,1,NaN,NaN,1.0,NaN,test
8097,0000003_as-0036229.jpg,NaN,1,NaN,NaN,1.0,NaN,train
5162,0000004_as-0036229.jpg,1,1,NaN,NaN,2.0,NaN,train
27605,0000005_as-0036229.jpg,1,NaN,NaN,NaN,1.0,NaN,train
60568,0000006_as-0036229.jpg,NaN,NaN,1,NaN,1.0,NaN,val
...,...,...,...,...,...,...,...,...
26156,0506218_sc-1037200.jpg,1,NaN,NaN,NaN,1.0,NaN,train
6733,0506219_sc-1037200.jpg,1,NaN,NaN,1,2.0,NaN,train
59626,0506220_sc-1037200.jpg,1,1,1,NaN,3.0,NaN,test
51588,0506225_sc-1023899.jpg,2,NaN,NaN,NaN,2.0,NaN,train
